#### Exponential Smoothing with Statesmodels

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings('ignore')

In [3]:
np.random.seed(42)
dates = pd.date_range(start='2012-01-01', end='2016-12-01', freq='MS')[:-1]

In [4]:
# Generate sales data (as above)
sales_data = []
for pid in range(1, 2851):
    num_obs = np.random.randint(10, 60)
    idx = np.random.choice(len(dates), num_obs, replace=False)
    sel_dates = dates[idx]
    base = np.random.uniform(50, 200)
    months = np.array([d.month - 1 for d in sel_dates])
    month_sin = np.sin(2 * np.pi * months / 12)
    month_cos = np.cos(2 * np.pi * months / 12)
    time = np.array([(d - dates[0]).days / 30.0 for d in sel_dates])
    sales = base + 0.5 * time + 15 * month_sin + 5 * month_cos + np.random.normal(0, 10, num_obs)
    sales = np.maximum(sales, 0)
    temp_df = pd.DataFrame({'product_id': pid, 'date': sel_dates, 'sales': sales})
    sales_data.append(temp_df)
df_sales = pd.concat(sales_data, ignore_index=True)

In [5]:
# Generate costs
df_costs = pd.DataFrame({
    'product_id': range(1, 2851),
    'unit_opportunity_loss': np.random.uniform(0.5, 5.0, 2850),
    'unit_inventory_cost': np.random.uniform(0.1, 2.0, 2850)
})

In [6]:
# Forecast
forecasts = {}
for pid in range(1, 2851):
    ts_df = df_sales[df_sales.product_id == pid].set_index('date').sort_index()['sales']
    if len(ts_df) < 12:
        forecasts[pid] = ts_df.mean() if len(ts_df) > 0 else 0
    else:
        try:
            model = ExponentialSmoothing(ts_df, trend='add', seasonal='add', seasonal_periods=12, damped_trend=True).fit(disp=False)
            pred = model.forecast(1)[0]
            forecasts[pid] = max(pred, 0)
        except:
            forecasts[pid] = ts_df.mean()
df_forecast = pd.DataFrame(list(forecasts.items()), columns=['product_id', 'forecast_dec2016'])

# Save/export as needed (e.g., df_forecast.to_csv('forecasts.csv'))
print(df_forecast.describe())

        product_id  forecast_dec2016
count  2850.000000       2850.000000
mean   1425.500000        140.183708
std     822.868459         43.236829
min       1.000000         60.654784
25%     713.250000        103.207737
50%    1425.500000        140.979247
75%    2137.750000        177.139646
max    2850.000000        222.318032


#### Linear and Random forest

In [7]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [9]:
np.random.seed(42)
dates = pd.date_range(start='2012-01-01', end='2016-12-01', freq='MS')[:-1]
n_months = len(dates)
n_products = 2850

In [10]:
# Generate sales data
sales_data = []
for pid in range(1, n_products + 1):
    num_obs = np.random.randint(10, 60)
    idx = np.random.choice(n_months, num_obs, replace=False)
    sel_dates = dates[idx]
    base = np.random.uniform(50, 200)
    months = np.array([d.month - 1 for d in sel_dates])
    month_sin = np.sin(2 * np.pi * months / 12)
    month_cos = np.cos(2 * np.pi * months / 12)
    time = np.array([(d - dates[0]).days / 30.0 for d in sel_dates])
    sales = base + 0.5 * time + 15 * month_sin + 5 * month_cos + np.random.normal(0, 10, num_obs)
    sales = np.maximum(sales, 0)
    temp_df = pd.DataFrame({'product_id': pid, 'date': sel_dates, 'sales': sales})
    sales_data.append(temp_df)
df_sales = pd.concat(sales_data, ignore_index=True)

In [11]:
# Generate costs
df_costs = pd.DataFrame({
    'product_id': range(1, n_products + 1),
    'unit_opportunity_loss': np.random.uniform(0.5, 5.0, n_products),
    'unit_inventory_cost': np.random.uniform(0.1, 2.0, n_products)
})

In [12]:
# Full TS
full_ts = []
for pid in range(1, n_products + 1):
    ts_df = df_sales[df_sales.product_id == pid].set_index('date').sort_index()
    full_series = pd.Series(index=dates, dtype=float)
    full_series.update(ts_df['sales'])
    full_ts.append(full_series)
ts_df_full = pd.DataFrame(full_ts).T
ts_df_full.index = dates
ts_df_full.columns = range(1, n_products + 1)

In [13]:
def create_features(ts):
    trend = np.arange(len(ts))
    month = ts.index.month
    month_sin = np.sin(2 * np.pi * (month - 1) / 12)
    month_cos = np.cos(2 * np.pi * (month - 1) / 12)
    lag1 = ts.shift(1)
    lag12 = ts.shift(12)
    return pd.DataFrame({'trend': trend, 'month_sin': month_sin, 'month_cos': month_cos, 'lag1': lag1, 'lag12': lag12}, index=ts.index)

In [14]:
# Actuals
actuals = {}
np.random.seed(42)
for pid in range(1, n_products + 1):
    last_sales = ts_df_full[pid].dropna().iloc[-1] if len(ts_df_full[pid].dropna()) > 0 else 0
    actual = max(last_sales + np.random.normal(0, 15), 0)
    actuals[pid] = actual
df_actual = pd.DataFrame(list(actuals.items()), columns=['product_id', 'actual_dec2016'])

In [15]:
# HW
forecasts_hw = {}
for pid in range(1, n_products + 1):
    ts_sparse = df_sales[df_sales.product_id == pid].set_index('date').sort_index()['sales']
    if len(ts_sparse) < 12:
        forecasts_hw[pid] = ts_sparse.mean() if len(ts_sparse) > 0 else 0
        continue
    try:
        ts_interp = ts_sparse.asfreq('MS').interpolate('linear')
        model = ExponentialSmoothing(ts_interp, trend='add', seasonal='add', seasonal_periods=12, damped_trend=True).fit(disp=False)
        pred = model.forecast(1)[0]
        forecasts_hw[pid] = max(pred, 0)
    except:
        forecasts_hw[pid] = ts_sparse.mean()
df_forecast_hw = pd.DataFrame(list(forecasts_hw.items()), columns=['product_id', 'forecast_dec2016'])

In [16]:
# LR
forecasts_lr = {}
for pid in range(1, n_products + 1):
    ts = ts_df_full[pid]
    feat = create_features(ts)
    y = ts
    mask = ~(y.isna() | feat['lag1'].isna() | feat['lag12'].isna())
    if mask.sum() < 3:
        forecasts_lr[pid] = y.dropna().mean() if len(y.dropna()) > 0 else 0
        continue
    X = feat.loc[mask, ['trend', 'month_sin', 'month_cos', 'lag1', 'lag12']].values
    y_train = y.loc[mask].values
    model = LinearRegression().fit(X, y_train)
    last_lag1 = ts.dropna().iloc[-1] if len(ts.dropna()) > 0 else 0
    lag12_val = ts['2015-12-01'] if not pd.isna(ts['2015-12-01']) else y.dropna().mean() if len(y.dropna()) > 0 else 0
    dec_X = np.array([[58, np.sin(2 * np.pi * 11 / 12), np.cos(2 * np.pi * 11 / 12), last_lag1, lag12_val]])
    pred = model.predict(dec_X)[0]
    forecasts_lr[pid] = max(pred, 0)
df_forecast_lr = pd.DataFrame(list(forecasts_lr.items()), columns=['product_id', 'forecast_dec2016'])

In [17]:
# RF
forecasts_rf = {}
for pid in range(1, n_products + 1):
    ts = ts_df_full[pid]
    feat = create_features(ts)
    y = ts
    mask = ~(y.isna() | feat['lag1'].isna() | feat['lag12'].isna())
    if mask.sum() < 5:
        forecasts_rf[pid] = y.dropna().mean() if len(y.dropna()) > 0 else 0
        continue
    X = feat.loc[mask, ['trend', 'month_sin', 'month_cos', 'lag1', 'lag12']].values
    y_train = y.loc[mask].values
    model = RandomForestRegressor(n_estimators=50, random_state=42).fit(X, y_train)
    last_lag1 = ts.dropna().iloc[-1] if len(ts.dropna()) > 0 else 0
    lag12_val = ts['2015-12-01'] if not pd.isna(ts['2015-12-01']) else y.dropna().mean() if len(y.dropna()) > 0 else 0
    dec_X = np.array([[58, np.sin(2 * np.pi * 11 / 12), np.cos(2 * np.pi * 11 / 12), last_lag1, lag12_val]])
    pred = model.predict(dec_X)[0]
    forecasts_rf[pid] = max(pred, 0)
df_forecast_rf = pd.DataFrame(list(forecasts_rf.items()), columns=['product_id', 'forecast_dec2016'])

In [23]:
from xgboost import XGBRegressor

# Inside product loop
forecasts_xgb = {}
for pid in range(1, n_products + 1):
    model = XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
    model.fit(X, y_train)
    pred = model.predict(dec_X)[0]
    forecasts_xgb[pid] = max(pred, 0)
df_forecast_xgb = pd.DataFrame(list(forecasts_xgb.items()), columns=['product_id', 'forecast_dec2016'])

In [24]:
# Evaluation
df_eval = df_actual.merge(df_forecast_hw.rename(columns={'forecast_dec2016': 'forecast_hw'}), on='product_id') \
                  .merge(df_forecast_lr.rename(columns={'forecast_dec2016': 'forecast_lr'}), on='product_id') \
                  .merge(df_forecast_rf.rename(columns={'forecast_dec2016': 'forecast_rf'}), on='product_id') \
                  .merge(df_forecast_xgb.rename(columns={'forecast_dec2016': 'forecast_xgb'}), on='product_id') \
                  .merge(df_costs, on='product_id')

In [27]:
def compute_loss(row, pred_col):
    actual = row['actual_dec2016']
    pred = row[pred_col]
    under = max(actual - pred, 0) * row['unit_opportunity_loss']
    over = max(pred - actual, 0) * row['unit_inventory_cost']
    return under + over

df_eval['loss_hw'] = df_eval.apply(lambda row: compute_loss(row, 'forecast_hw'), axis=1)
df_eval['loss_lr'] = df_eval.apply(lambda row: compute_loss(row, 'forecast_lr'), axis=1)
df_eval['loss_rf'] = df_eval.apply(lambda row: compute_loss(row, 'forecast_rf'), axis=1)
df_eval['loss_xgb'] = df_eval.apply(lambda row: compute_loss(row, 'forecast_xgb'), axis=1)

print(df_forecast_hw.describe())  # HW summary
print(df_forecast_lr.describe())  # LR
print(df_forecast_rf.describe())  # RF
print(df_forecast_xgb.describe()) #xgb
loss_summary = pd.DataFrame({
    'Model': ['HW', 'LR', 'RF', 'XGB'],
    'Total Loss': [df_eval['loss_hw'].sum(), df_eval['loss_lr'].sum(), df_eval['loss_rf'].sum(), df_eval['loss_xgb'].sum()],
    'Average Loss': [df_eval['loss_hw'].mean(), df_eval['loss_lr'].mean(), df_eval['loss_rf'].mean(), df_eval['loss_xgb'].mean()]
})
print(loss_summary)
print(df_eval[['product_id', 'actual_dec2016', 'forecast_hw', 'forecast_lr', 'forecast_rf', 'forecast_xgb', 'loss_hw', 'loss_lr', 'loss_rf', 'loss_xgb']].head(10))

        product_id  forecast_dec2016
count  2850.000000       2850.000000
mean   1425.500000        140.183708
std     822.868459         43.236829
min       1.000000         60.654784
25%     713.250000        103.207737
50%    1425.500000        140.979247
75%    2137.750000        177.139646
max    2850.000000        222.318032
        product_id  forecast_dec2016
count  2850.000000       2850.000000
mean   1425.500000        149.071744
std     822.868459         53.555884
min       1.000000          0.000000
25%     713.250000        109.603625
50%    1425.500000        149.270540
75%    2137.750000        186.456757
max    2850.000000        791.061975
        product_id  forecast_dec2016
count  2850.000000       2850.000000
mean   1425.500000        143.780526
std     822.868459         43.754925
min       1.000000         50.851755
25%     713.250000        106.403867
50%    1425.500000        143.890380
75%    2137.750000        181.406681
max    2850.000000        239.954998
 

#### PyTorch LSTM

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

In [36]:
# Assume df_sales, dates, n_products=2850, ts_df_full from prior code
# (full 59-month series per product with NaNs; regenerate if needed as in earlier snippets)

def create_features(ts):
    """Create multivariate features: trend, month_sin, month_cos, lag1, lag12"""
    trend = np.arange(len(ts))
    month = ts.index.month.values
    month_sin = np.sin(2 * np.pi * (month - 1) / 12)
    month_cos = np.cos(2 * np.pi * (month - 1) / 12)
    lag1 = ts.shift(1, fill_value=ts.mean())
    lag12 = ts.shift(12, fill_value=ts.mean())
    features = pd.DataFrame({
        'trend': trend, 'month_sin': month_sin, 'month_cos': month_cos,
        'lag1': lag1, 'lag12': lag12
    }, index=ts.index)
    return features

class TimeSeriesDataset(Dataset):
    def __init__(self, features, target, seq_len=12):
        self.features = features.values  # (n_timesteps, 5)
        self.target = target.values  # (n_timesteps,)
        self.seq_len = seq_len

    def __len__(self):
        return len(self.target) - self.seq_len

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.seq_len]  # Fixed length (seq_len, 5)
        y = self.target[idx + self.seq_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class LSTMPredictor(nn.Module):
    def __init__(self, input_size=5, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        dropout_val = dropout if num_layers > 1 else 0
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_val)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1]).squeeze(1)  # (batch,)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Forecasts dict
forecasts_lstm = {}
for pid in range(1, n_products + 1):
    ts = ts_df_full[pid]  # Series with NaNs
    num_obs = len(ts.dropna())
    if num_obs < 12:  # Fallback for too sparse
        forecasts_lstm[pid] = ts.mean()
        continue
    
    # Fill missing values for consecutive sequences
    ts_filled = ts.interpolate(method='linear').fillna(ts.mean())
    feat_df = create_features(ts_filled)
    
    # Dataset (fixed length, no NaNs)
    dataset = TimeSeriesDataset(feat_df, ts_filled, seq_len=12)
    loader = DataLoader(dataset, batch_size=8, shuffle=True)  # Small batch for efficiency
    
    # Model
    model = LSTMPredictor(input_size=5).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # Train
    model.train()
    for epoch in range(50):
        epoch_loss = 0
        for x, y in loader:
            x, y = x.to(device), y.to(device).unsqueeze(1)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
    
    # Predict Dec 2016 (extrapolate from last 12 months)
    model.eval()
    last_feat = feat_df.iloc[-12:].values  # (12, 5)
    with torch.no_grad():
        last_seq = torch.tensor(last_feat, dtype=torch.float32).unsqueeze(0).to(device)  # (1, 12, 5)
        pred = model(last_seq).item()
    forecasts_lstm[pid] = max(pred, 0)

df_forecast_lstm = pd.DataFrame(list(forecasts_lstm.items()), columns=['product_id', 'forecast_dec2016'])

# Evaluation (reuse compute_loss from prior; merge with df_actual, df_costs)
df_eval_lstm = df_actual.merge(df_forecast_lstm, on='product_id').merge(df_costs, on='product_id')
df_eval_lstm['loss_lstm'] = df_eval_lstm.apply(lambda row: compute_loss(row, 'forecast_dec2016'), axis=1)
print("LSTM Total Loss:", df_eval_lstm['loss_lstm'].sum())
print(df_forecast_lstm.describe())

LSTM Total Loss: 987650.2074498702
        product_id  forecast_dec2016
count  2850.000000       2850.000000
mean   1425.500000         18.329829
std     822.868459         26.236903
min       1.000000          5.620905
25%     713.250000         11.390396
50%    1425.500000         13.529449
75%    2137.750000         15.959415
max    2850.000000        220.170671


#### Prophet Example

In [37]:
import pandas as pd
import numpy as np
from prophet import Prophet
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')  # Suppress Prophet warnings for clean output

# Step 1: Generate fake historical sales data (sparse)
np.random.seed(42)  # For reproducibility
n_products = 10  # Use 10 for demo; set to 2850 for full
dates = pd.date_range(start='2012-01-01', end='2016-11-01', freq='MS')  # 59 months
sales_data = []

for prod_id in range(1, n_products + 1):
    # Simulate sparsity: random 20-50 observations per product
    n_obs = np.random.randint(20, 50)
    selected_dates = np.random.choice(dates, size=n_obs, replace=False)
    # Fake order amounts: base 100 + seasonal sin wave + noise (lognormal for positivity)
    months = np.arange(len(dates))
    seasonal = 20 * np.sin(2 * np.pi * months / 12)  # Annual seasonality
    base = 100 + seasonal[:n_obs]  # Align to selected dates
    order_amounts = np.random.lognormal(mean=np.log(base), sigma=0.3)
    for date, amount in zip(selected_dates, order_amounts):
        sales_data.append({'product_id': prod_id, 'ds': date, 'y': amount})

df_sales = pd.DataFrame(sales_data)
print(f"Generated {len(df_sales)} historical observations across {n_products} products.")

# Step 2: Generate fake costs data
costs_data = {
    'product_id': range(1, n_products + 1),
    'unit_opportunity_loss': np.random.uniform(1, 10, n_products),  # cu
    'unit_inventory_cost': np.random.uniform(1, 10, n_products)     # co
}
df_costs = pd.DataFrame(costs_data)
print(df_costs.head())

# Step 3: Forecast for December 2016
forecasts = []
dec_2016 = pd.to_datetime('2016-12-01')

for prod_id in df_sales['product_id'].unique():
    # Historical data for this product
    hist = df_sales[df_sales['product_id'] == prod_id][['ds', 'y']].sort_values('ds')
    
    if len(hist) == 0:
        # Handle no data: use global mean (or skip/error in production)
        global_mean = df_sales['y'].mean()
        forecasts.append({'product_id': prod_id, 'forecast': global_mean})
        continue
    
    # Fit Prophet
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    m.fit(hist)
    
    # Make future dataframe for Dec 2016
    future = m.make_future_dataframe(periods=1, freq='MS', include_history=False)
    fc = m.predict(future)
    
    # Extract forecast components
    yhat = fc['yhat'].iloc[-1]
    yhat_lower = fc['yhat_lower'].iloc[-1]
    yhat_upper = fc['yhat_upper'].iloc[-1]
    
    # Approximate sigma from 80% PI (z=1.282 for 80%)
    sigma = (yhat_upper - yhat_lower) / (2 * 1.282)
    sigma = max(sigma, 0.01)  # Avoid zero/negative sigma
    
    # Get costs for alpha
    prod_costs = df_costs[df_costs['product_id'] == prod_id].iloc[0]
    cu = prod_costs['unit_opportunity_loss']
    co = prod_costs['unit_inventory_cost']
    alpha = cu / (cu + co)
    
    # α-quantile forecast
    z_alpha = norm.ppf(alpha)
    quantile_forecast = yhat + z_alpha * sigma
    quantile_forecast = max(quantile_forecast, 0)  # Ensure non-negative
    
    forecasts.append({
        'product_id': prod_id,
        'forecast': quantile_forecast,
        'alpha': alpha,
        'yhat_mean': yhat,
        'sigma': sigma
    })

# Step 4: Output results
df_forecasts = pd.DataFrame(forecasts)
print("\nForecasts for December 2016:")
print(df_forecasts.round(2))

# For full scale (2850 products), the loop scales well; add parallelization if needed
# e.g., from joblib import Parallel, delayed; Parallel(n_jobs=-1)(delayed(forecast_func)(prod) for prod in products)

Generated 312 historical observations across 10 products.
   product_id  unit_opportunity_loss  unit_inventory_cost
0           1               3.802998             8.771663
1           2               3.046561             9.306642
2           3               6.471050             5.191301
3           4               4.413751             5.327533
4           5               7.698247             9.266093


22:03:00 - cmdstanpy - INFO - Chain [1] start processing
22:03:00 - cmdstanpy - INFO - Chain [1] done processing
22:03:00 - cmdstanpy - INFO - Chain [1] start processing
22:03:01 - cmdstanpy - INFO - Chain [1] done processing
22:03:01 - cmdstanpy - INFO - Chain [1] start processing
22:03:01 - cmdstanpy - INFO - Chain [1] done processing
22:03:02 - cmdstanpy - INFO - Chain [1] start processing
22:03:02 - cmdstanpy - INFO - Chain [1] done processing
22:03:02 - cmdstanpy - INFO - Chain [1] start processing
22:03:03 - cmdstanpy - INFO - Chain [1] done processing
22:03:03 - cmdstanpy - INFO - Chain [1] start processing
22:03:04 - cmdstanpy - INFO - Chain [1] done processing
22:03:04 - cmdstanpy - INFO - Chain [1] start processing
22:03:04 - cmdstanpy - INFO - Chain [1] done processing
22:03:05 - cmdstanpy - INFO - Chain [1] start processing
22:03:05 - cmdstanpy - INFO - Chain [1] done processing
22:03:05 - cmdstanpy - INFO - Chain [1] start processing
22:03:07 - cmdstanpy - INFO - Chain [1]


Forecasts for December 2016:
   product_id  forecast  alpha  yhat_mean  sigma
0           1      0.00   0.30   -1352.70  11.24
1           2    158.54   0.25     165.30   9.86
2           3     87.74   0.55      85.93  13.12
3           4    121.78   0.45     124.48  22.92
4           5    118.09   0.45     120.41  19.96
5           6    111.23   0.31     122.54  23.08
6           7      0.00   0.86    -143.87  19.67
7           8     41.69   0.41      46.12  19.74
8           9     55.45   0.39      63.75  28.51
9          10     82.92   0.43      87.14  24.96
